In [1]:
import ollama

# pull llama2
ollama.pull("mistral")
# ollama.pull("mixtral")  # mistral is the default model, use different.
ollama.pull("codellama") # TODO: maybe use larger model, specific model for math.
# maybe use https://ollama.com/library/wizard-math for math-related stuff in parallel with codellama (can use 34b)
# or maybe use https://ollama.com/solobsd/llemma-7b for math-related stuff in parallel with codellama (only can use 7b)

In [ ]:
import pandas as pd

kg_data = {
    "Concept": ["derivative", "integral", "plot"],
    "CodePattern": [
        "Use the differentiation operator in sympy.",
        "Use the integration operator in sympy.",
        "Use matplotlib to plot the function.",
    ],
}
kg = pd.DataFrame(kg_data)

In [ ]:
# todo: maybe use an llm here?
def lookup_code_pattern(query):
    keywords = ["derivative", "integral", "plot"]
    for keyword in keywords:
        if keyword in query.lower():
            code_pattern = kg[kg["Concept"] == keyword]["CodePattern"].iloc[0]
    return keyword, code_pattern

In [ ]:
# now, call this knowledge to ollama's codellama solution.
# todo: use instruct version instead of 7b-base
def generate_code_llm(query):
    concept, code_pattern = lookup_code_pattern(query)
    query_appended = query + "Here's a hint:" + code_pattern

    code_query = f"You are an expert programmer that writes simple, concise code in Python. {query_appended} ONLY the code is needed,Absolutely NO explanation. Use SymPy."

    print("query sent to codellama: ", code_query)
    response = ollama.chat(
        "codellama", messages=[{"role": "system", "content": code_query}]
    )
    resp = response["message"]["content"].replace("```", "")
    print("response from codellama:\n", resp)
    attempt = 5
    while attempt > 0:
        try:
            code_result = exec(resp)
            break
        except Exception as e:
            if attempt == 1:
                continue
            error = f"Error in code execution: {e}"
            code_query += f"\nYou gave this response.\n{resp}\nHowever, execution of the response resulted in an error.\n{error}\nPlease correct the error and try again. ONLY the corrected code is needed,Absolutely NO explanation. Use SymPy."
            print("query sent to codellama: ", code_query)
            response = ollama.chat(
                "codellama", messages=[{"role": "system", "content": code_query}]
            )
            resp = response["message"]["content"].replace("```", "")
            print("response from codellama:\n", resp)
            attempt -= 1

    # todo: let answer llm know if the code generated by code LLM was executable
    print("code generated by codellama: ", code_result)
    return code_result

In [ ]:
def generate_llm_response(query):
    concept, context_info = lookup_code_pattern(query)
    # code result
    code_result = generate_code_llm(query)
    if concept:
        response = ollama.chat(
            model="mistral",
            messages=[
                {
                    "role": "system",
                    "content": f"Answer the question {query} using the answer {code_result}. Just provide the answer.",
                }
            ],
        )
    answer = response["message"]["content"]
    return answer

In [ ]:
query = "Compute the derivative of x^2 with respect to x."
print(generate_llm_response(query))

In [ ]:
query2 = "Compute the integral of x^2 with respect to x."
print(generate_llm_response(query2))